## Exploratory Data Analysis and Predictive Modelling of Local Atmospheric Pollutants

###### Kaggle Tabular Playground Series - July 2021

The given dataset includes dates, temperatures, humidities (relative and absolute), and data from sensors.

The goal of the project is to accurately predict the amounts of three target variables (pollutants);
target_carbon_monoxide, target_benzene, and target_nitrogen_oxides.

An additional personal goal is to determine the factors which effect each of the target variables the most.